## Problema 1:
Determine los desplazamientos de los bordes libres y los esfuerzos en función de la posición para la
placa empotrada mostrada en la figura. Tome las dos particiones en elementos mostradas debajo y
compare los resultados. Asuma que E = 30 x 106 psi y n = 0.30.


In [1]:
import numpy as np

#Datos
E = 30e6  #[psi]
v = 0.3   #Relacion poison
espesor = 1   #[inch]

T_dist = 1000   #[psi] Tension aplicada distribuida

#Caso 1:
'''
MN = np.array([[0, 0],
               [0, 10],
               [20, 10],
               [20, 0]
              ])

MC = np.array([[0, 1, 2],     #Armo los triangulos
               [0, 2, 3]
              ])

MNE = np.array([ [0,0,0],      #Matriz de nodos empotrados [nodo, ux, uy]
                 [1,0,0] ])
MNT = np.array([ [2,5000,0],   #Matriz de nodos traccionados [nodo, Fx, Fy]
                 [3,5000,0] ])
'''
##--------------------------------------------------##

#Caso 2:
MN = np.array([[0, 0],
               [0, 10],
               [20, 10],
               [20, 0],
               [10, 5]
              ])

MC = np.array([[0, 1, 4],     #Armo los triangulos
               [1, 2, 4],
               [2, 3, 4],
               [0, 3, 4]
              ])

MNE = np.array([ [0,0,0],      #Matriz de nodos empotrados [nodo, ux, uy]
                 [1,0,0] ])
MNT = np.array([ [2,5000,0],   #Matriz de nodos traccionados [nodo, Fx, Fy]
                 [3,5000,0] ])

#---------------------------------------###

cant_nodos = len(MN[:])            #Cantidad de nodos en el problema
cant_elem = len(MC[:])
glxn = 2                           #Grados de libertad por nodo


#Vectores fuerza y desplazamiento
u = np.zeros(2*cant_nodos)
F = np.zeros(2*cant_nodos)

#Matrices del problema
Matriz_D = (E/(1-v**2))*np.array([[1, v, 0], [v, 1, 0], [0, 0, 0.5*(1-v)]])

Matriz_K = np.zeros([glxn*cant_nodos, glxn*cant_nodos])   #Matriz global
Matriz_coor = np.ones((3,3)) 
Matriz_loc_B = []
areas = []


In [8]:
#Utilizando Gmsh para el mallado

import numpy as np
import gmsh
import sys

gmsh.initialize()
gmsh.model.add('chapa1')

#Agrego los puntos
lc = 1       #Factor de escala
L = 10
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc)
p2 = gmsh.model.geo.addPoint(0, L, 0, lc)
p3 = gmsh.model.geo.addPoint(2*L, L, 0, lc)
p4 = gmsh.model.geo.addPoint(2*L, 0, 0, lc)

#Armo las lineas
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])    #Genera el contorno, figura cerrada. lineas en orden

S1 = gmsh.model.geo.addPlaneSurface([C1]) #Genero la superficie delimitada por el contorno
gmsh.model.geo.synchronize()   #Sincronizamos lo cargado

#gmsh.fltk.run()

#Defino los grupos fisicos:
Empotrado = gmsh.model.addPhysicalGroup(1, [l4])  #dimension 1, linea 4 --> grupo fisico
gmsh.model.setPhysicalName(1, Empotrado, 'Empotrado')   #Se le pone un nombre o categoria
Traccionado = gmsh.model.addPhysicalGroup(1, [l1])
gmsh.model.setPhysicalName(1, Traccionado, 'Traccionado')
Superficie = gmsh.model.addPhysicalGroup(2, [S1])
gmsh.model.setPhysicalName(2, Superficie, 'Superficie')

gmsh.model.mesh.generate(2)

gmsh.fltk.run()

#Recupero MC y MN del mallado:
NodeInfo = gmsh.model.mesh.get_nodes()   #Devuelve una tupla con los nodos NodeInfo[0], las posiciones NodeInfo[1] (MN)
NumeroNodos = NodeInfo[0].shape[0]

MN = NodeInfo[1].reshape(NumeroNodos , 3)

ElementInfo = gmsh.model.mesh.get_elements()    #Recupera los elementos 
#ElementInfo   #tupla con 3 elementos: [0]:tipos de elementos 1:lineas, 2:triangulos, 15:puntos, [1]: etiquetas de 
              #los distintos elementos len=3, [2]: Matrices de conectividad

ETYPES = ElementInfo[0]
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)  #Este metodo toma solo los triangulos (2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])

#Condiciones de contorno:

entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado)
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])
Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2)    #Con los elementos líneas traccionadas podemos calcular las longitudes y distribuir la fuerza externa.

Longitudes = np.abs( MN[Ltraccionada[:,0]-1,1] - MN[Ltraccionada[:,1]-1,1] )

#Calculo las fuerzas distribuidas
fuerza = np.zeros((2*NumeroNodos,1))
espesor = 1
tension = 1000 #Pa

for l, linea in enumerate(Ltraccionada):
    Flocal = np.array([[1],[1]])*tension*espesor*Longitudes[l]
    n1 = linea[0]
    n2 = linea[1]
    #print(Flocal)
    fuerza[ np.array([2*(n1-1), 2*(n2-1)], dtype=int)] += Flocal

#Nodos empotrados
NodosEmpotrados = gmsh.model.mesh.get_nodes_for_physical_group(1,Empotrado)  #Para los empotrados no necesito las lineas, solo los nodos
                    #NodosEmpotrados[0] --> Indices de los nodos empotrados
MNE = NodosEmpotrados[1].reshape((NodosEmpotrados[0].shape[0], 3))    #Posiciones x, y, z de los nodos empotrados. x=0

#Defino vectores r y s:
s = []
r = np.arange(2*NumeroNodos)    #Cargo todos los nodos (*2) para los indices de la matriz_global
for n, nodo in enumerate(NodosEmpotrados[0]):
    s.append(
        np.linspace(2*nodo, 2*nodo +1, 2)
    )
s = np.array(s).astype(int)
r = np.delete( r, s )       #Saco de r todo lo que puse en s


cant_nodos = NumeroNodos            #Cantidad de nodos en el problema
cant_elem = len(MC[:])
glxn = 2                           #Grados de libertad por nodo

MNT = 





In [2]:
for e in range(cant_elem):
    i, m, j = MC[e]    #Nodos de cada elemento

    x = np.array([MN[i,0],MN[m,0],MN[j,0]])  #coordenada x de cada elemento
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])  #coordenada y de cada elemento
   
    #Ai,j,m = 0 luego de derivar las Ni
    Bi = y[2] - y[1]; Bj = y[1] - y[0]; Bm = y[0] - y[2]
    Gi = x[1] - x[2]; Gj = x[0] - x[1]; Gm = x[2] - x[0]
    #print(Bi, Bj, Bm)
    #print(Gi, Gj, Gm)
    
    #armo la matriz de coordenadas
    Matriz_coor[:,1] = x
    Matriz_coor[:,2] = y
    
    area = np.linalg.det(Matriz_coor)/2   #Obtengo A
    areas.append(area)
    
    #Armo matriz para las deformaciones (deriv de u), derivadas de funcion interp (N_i,j,m):
    Matriz_B = (1/(2*abs(area)))*np.array([[Bi, 0, Bm, 0, Bj, 0], [0, Gi, 0, Gm, 0, Gj], [Gi, Bi, Gm, Bm, Gj, Bj]])   
    Matriz_loc_B.append(Matriz_B)
    
    #Armo la matriz global
    Matriz_local = espesor * abs(area) * (np.dot(np.transpose(Matriz_B), np.dot(Matriz_D, Matriz_B)))
    ind_glob = np.array([2*i, 2*i+1, 2*m, 2*m+1, 2*j, 2*j+1])  #Posiciones en la matriz global (elementos 0, 1, 2 y 3)
    Matriz_K[np.ix_(ind_glob, ind_glob)] += Matriz_local


In [3]:
#Condiciones de contorno:

#d0x = d0y = d1x = d1y = 0  (lado empotrado)
#F2y = F3y = 0; F2x = F3x = 5000

desplaz = np.zeros(cant_nodos*2)   #Vector con los desplazamiento x, y de todos los nodos
fuerza = []    #Fuerzas conocidas
nodos = []     #Nodos en los que conozco la fuerza pero no desplazamiento, (carga el vector desplaz)

#Necesito los indices de los desp conocidos para reducir la matriz global
for i in MNT:
    nodo, valx, valy = i
    nodos.append(2*nodo)    #Nodos para calcular los desplazamientos
    nodos.append(2*nodo+1)
    fuerza.append(valx)
    fuerza.append(valy)
    
for i in MNE:
    nodo, valx, valy = i
    desplaz[2*nodo] = valx  #Cargo los nodos cuyo desplazamiento ya conozco
    desplaz[2*nodo+1] = valy

#CALCULO LOS DESPLAZAMIENTOS:
#Matriz reducida

Matriz_red = Matriz_K[np.ix_(nodos, nodos)]
fuerza = np.array(fuerza).reshape(-1,1)

desplaz_incog = np.linalg.solve(Matriz_red, fuerza)     

for i, n in enumerate(nodos):
    desplaz[n] = desplaz_incog[i]

#CALCULO LAS FUERZAS:

fuerza_tot = np.dot(Matriz_K, desplaz).round(4)


In [4]:
#TENSIONES
tensiones = []
for e in range(cant_elem):
    i, m, j = MC[e]
    
    #Selecciono los desplazamientos de los nodos de cada elemento
    desp_elem = np.array([[desplaz[2*i], desplaz[2*i+1],desplaz[2*m], desplaz[2*m+1],desplaz[2*j], desplaz[2*j+1]]])  
    tension = np.dot(Matriz_D, np.dot(Matriz_loc_B[e], desp_elem.reshape(-1,1)))
    tensiones.append(tension)